## Hi Marta! 
#### This is a quick script to show you how to pull data from nxcals using Swan. 
#### Before running the code make sure to select the 'star' symbol in the tab and select include 'nxcals' and 'sparcmetrics' options.

In [1]:
# some packages for visualisation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import tfs

In [2]:
import sys
sys.path.append('/eos/home-i03/m/morwat/.local/lib/python3.9/site-packages/')

In [4]:
# import package to access Timber
import pytimber

# initialise the logging
ldb = pytimber.LoggingDB(spark_session=spark)

In [5]:
# you can pull data from a fill you are interested in
fill = 8635

# get the timestamps of different beam modes during chosen fill
fill_timestamps = ldb.get_lhc_fill_data(fill)
ffn = {item['mode']:item for item in fill_timestamps['beamModes']}

/cvmfs/sft.cern.ch/lcg/views/LCG_105a_nxcals_pro/x86_64-el9-gcc13-opt/lib/python3.9/site-packages/pytimber/utils.py:93: UserWarning: Discarding nonzero nanoseconds in conversion.
  return pd.to_datetime(nanos, unit="ns", utc=utc).to_pydatetime()


In [6]:
# define times of the mode you are interested in exploring
t1 = ffn['INJPHYS']['startTime']
t2 = ffn['INJPHYS']['endTime']

In [7]:
print(fill_timestamps)

{'fillNumber': 8635, 'startTime': 1682072342.149488, 'endTime': 1682077145.473988, 'beamModes': [{'mode': 'INJPROB', 'startTime': 1682072342.149488, 'endTime': 1682072350.033238}, {'mode': 'INJPROB', 'startTime': 1682072350.033238, 'endTime': 1682072797.443238}, {'mode': 'INJPHYS', 'startTime': 1682072797.443238, 'endTime': 1682074536.350363}, {'mode': 'PRERAMP', 'startTime': 1682074536.350363, 'endTime': 1682075157.375738}, {'mode': 'RAMP', 'startTime': 1682075157.375738, 'endTime': 1682076435.842238}, {'mode': 'FLATTOP', 'startTime': 1682076435.842238, 'endTime': 1682076963.778363}, {'mode': 'BEAMDUMP', 'startTime': 1682076963.778363, 'endTime': 1682076965.550363}, {'mode': 'RAMPDOWN', 'startTime': 1682076965.550363, 'endTime': 1682077145.473988}]}


In [8]:
# load the variable you are interested in
BPM_varname_H = 'BFC.LHC:OrbitAcq:positionsH'
BPM_varname_V = 'BFC.LHC:OrbitAcq:positionsV'

BPM_pos_H = ldb.get(BPM_varname_H, t1, t2)
BPM_pos_V = ldb.get(BPM_varname_V, t1, t2)

In [32]:
BPM_names = ldb.get('BFC.LHC:Mappings:fBPMNames_h', t1, t2)
BPM_names_np = BPM_names['BFC.LHC:Mappings:fBPMNames_h'][1][0]

In [10]:
# Convert Unix time to readable date-time format
readable_times = np.datetime64('1970-01-01') + np.timedelta64(1, 's') * BPM_pos_H['BFC.LHC:OrbitAcq:positionsH'][0]

In [11]:
# Target date-time to find
target_time = np.datetime64('2023-04-21T10:53:15')

# Use np.where to find the indices
index = np.where(readable_times == target_time)

In [12]:
# the first array is time, and the second is an array of arrays containing readings from different BPMs
t = BPM_pos_H['BFC.LHC:OrbitAcq:positionsH'][0]
BPM_readings_H = BPM_pos_H['BFC.LHC:OrbitAcq:positionsH'][1]

In [13]:
# let's turn the arrays into DataFrame so we can easily access it
df = pd.DataFrame(bpm for bpm in BPM_readings_H)

In [63]:
df1 = tfs.read('/eos/project-c/collimation-team/machine_configurations/LHC_run3/2023/MADX_thick/injection/all_optics_B1.tfs')
df1 = df1[df1.KEYWORD.str.contains('MONITOR')]
df1 = df1[~df1.NAME.str.contains('DOROS')]
filtered_df1 = df1[df1['NAME'].isin(BPM_names_np)]
filtered_df1 = filtered_df1.reset_index(drop=True)

x = df.iloc[index].T
filtered_df1['BPMx'] = x
filtered_df1 = filtered_df1.dropna()

beam1 = pd.DataFrame({'NAME': filtered_df1.NAME,
                        'S': filtered_df1.S,
                        'X': filtered_df1.BPMx})

In [66]:
# now I am plotting all BPM readings in the same plot, but you can later on extract just the one you are interested in
fig = go.Figure()

fig.add_trace(go.Scatter(x=beam1.S, y=beam1.X, mode='lines', text = beam1.NAME))

fig.show()

In [ ]:
# now I am plotting all BPM readings in the same plot, but you can later on extract just the one you are interested in
fig = go.Figure()

for i in range(542, 1084):
    fig.add_trace(go.Scatter(x=[i], y=[-df[i][index]], mode='markers', name = str(BPM['BFC.LHC:Mappings:fBPMNames_h'][1][0][i])))

fig.show()